<a href="https://colab.research.google.com/github/ThomasAlbin/Astroniz-YT-Tutorials/blob/main/CompressedCosmos/CompressedCosmos_MarsSunrise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Sun from Mars

In this script we are taking a look on how to compute the Sun's position as see from the surface of Mars. For more details, please refer to my YouTube channel: https://www.youtube.com/@Astroniz

In [ ]:
!pip install spiceypy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.1 MB/s eta 0:00:00


In [ ]:
import spiceypy
import numpy as np
import math
import os
from datetime import datetime

# Define kernel file names
# Ensure these files are present!!!
LASKEEP_KERNEL = "naif0012.tls"
PCK_KERNEL = "pck00011.tpc"
SPK_KERNEL = "de442s.bsp"
kernel_paths = [LASKEEP_KERNEL, PCK_KERNEL, SPK_KERNEL]

# --- Load Kernels ---
for kp in kernel_paths:
    spiceypy.furnsh(kp)

# SPICE Constants
MARS_ID = 4 # Barycenter (!)
MARS_NAME = 'MARS'
SUN_ID = 10
J2000_FRAME = 'J2000'
MARS_FIXED_FRAME = 'IAU_MARS'
ABERRATION_CORRECTION = 'LT+S'

In [ ]:
# Coordiantes & Date-Time
# Example: Location near Gale Crater (Curiosity Rover landing site approx)
mars_latitude = -5.0  # Degrees North
mars_longitude = 137.0 # Degrees East

datetime_utc = datetime(2025, 4, 6, 15, 16, 31)
observation_utc = datetime_utc.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
# Convert input angular measures to radians for SPICE functions
mars_latitude_rad = math.radians(mars_latitude)
mars_longitude_rad = math.radians(mars_longitude)

# ET
et = spiceypy.utc2et(observation_utc) # Convert UTC to Ephemeris Time

In [ ]:
# Obtain now geo-physical parameter of Mars
radii_km = spiceypy.bodvrd(MARS_NAME, 'RADII', 3)
equatorial_radius_km = radii_km[1][0]
polar_radius_km = radii_km[1][2]

# We need to determine how "flat" Mars is for a future SPICE function called GEOREC
flattening = (equatorial_radius_km - polar_radius_km) / equatorial_radius_km

In [ ]:
# Observer Position (Cartesian in IAU_MARS)
pos_observer_mars_fixed = spiceypy.georec(mars_longitude_rad,
                                          mars_latitude_rad,
                                          0,
                                          equatorial_radius_km,
                                          flattening)

# Compute vector Mars Center to Sun Center in IAU_Mars
mars2sun_vec, _ = spiceypy.spkgps(targ=SUN_ID,
                                  et=et,
                                  ref=MARS_FIXED_FRAME,
                                  obs=MARS_ID)

# Observer to Sun is: Mars2Sun - Mars2Obs
obs2sun_vec = mars2sun_vec - pos_observer_mars_fixed

In [ ]:
# Now we can compute the RA/DEC values of the Sun and teh Alt/AZ values as seen from the oberserver
_, sun_ra_rad, sun_dec_rad = spiceypy.recrad(mars2sun_vec)
_ sun_az_rad, sun_elev_rad = spiceypy.recazl(obs2sun_vec,
                                              mars_latitude_rad,
                                              mars_longitude_rad)

In [ ]:
print(f"Apparent Sun RA:    {np.degrees(sun_ra_rad):.1f} degrees")
print(f"Apparent Sun Dec:   {np.degrees(sun_dec_rad):.1f} degrees")
print(f"Apparent Sun Az:    {np.degrees(sun_az_rad):.1f} degrees (from North, clockwise)")
print(f"Apparent Sun Alt:   {np.degrees(sun_elev_rad):.1f} degrees (above horizon)")